In [1]:
#Import Standard Libraries
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import numpy as np
%matplotlib inline

# Custom Function and Transformers

In [2]:
#Function to Import Data

def import_data(path, drop, sep,dropna=True, drop_duplicates=True):
    data = pd.read_csv(path, sep)
    print("Banyaknya baris dan kolom", data.shape)
    print("Banyaknya data duplicate", data.duplicated().sum())
    data = data.drop_duplicates() if drop_duplicates else data
    data = data.dropna() if dropna else data
    data = data.drop(drop,axis=1)
    print("Banyaknya data setelah di drop", data.shape)
    #print(data.head())
    return data

In [3]:
#Function to Extract Input and Output

def extract_input_output(data, output):
    y = data[output]
    x = data.drop(output, axis=1)
    #print(x.columns) #optional
    #print(y.head()) #optional
    return x,y

In [4]:
#function to split train test data (default test_size = 0.4)
from sklearn.model_selection import train_test_split

def train_test_data(x,y, _test_size = 0.4): 
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size =_test_size,
                                                        random_state =100)
   # print(x_train.head(), y_train.head()) #
    #print(x_test.head(), y_test.head()) #
    return x_train, x_test, y_train, y_test

In [5]:
#Function to evaluate score
from sklearn.metrics import accuracy_score

def scoring(model) :
    y_predict = model.predict(x_train)
    print("Model Score based on training data")
    print(accuracy_score(y_train, y_predict))
    y_predict = model.predict(x_test)
    print("Model Score based on test data")
    print(accuracy_score(y_test, y_predict))


In [6]:
from sklearn.base import TransformerMixin #gives fit_transform method for free
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
  def __init__(self, attribute_names):
    self.attribute_names = attribute_names
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    return X[self.attribute_names].values

# Visualization Functions

In [8]:
#Make sure y_train is balanced
def plot_y_balance(y, df) : 
  ax = sns.countplot(y,label="Count")       # M = 212, B = 357
  C, N = y.value_counts()
  print('Number of Churned: ',C)
  print('Number of Not Churned : ',N) 
  df.isChurned.value_counts(normalize=True)

# Pre Processing Pipeline

In [9]:
df = pd.read_csv('../Bagian C/datasets/iris.csv')

In [10]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [11]:
df[' species']

0          setosa
1          setosa
2          setosa
3          setosa
4          setosa
          ...    
145     virginica
146     virginica
147     virginica
148     virginica
149     virginica
Name:  species, Length: 150, dtype: object

In [12]:
x, y = extract_input_output(df, ' species')

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import FeatureUnion

num_attribs = list(x)


preprocessor = Pipeline([
#     ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="median")),
    #('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
  ])

# cat_pipeline = Pipeline([
#     ('selector', DataFrameSelector(cat_attribs)),
#     ('label_binarizer', MyLabelBinarizer()),
#   ])

# preprocessor = FeatureUnion(transformer_list=[
#     ("num_pipeline", num_pipeline),
#     ("cat_pipeline", cat_pipeline),
#   ])

# Machine Learning Pipeline

In [15]:
from clf.mlp import MyMlp
from clf.utils import read_csv, oneHotEncoder, scale_data


In [16]:
input_layer = len(list(x))
print(input_layer)
output_layer = len(set(y))
print(output_layer)
hidden_layer = [4, 3]
print(hidden_layer)

4
3
[4, 3]


In [17]:
myclf = MyMlp(input_layer, hidden_layer, output_layer)

In [18]:
y_encoded = oneHotEncoder(y)

In [19]:
myclf.fit(x.values,y_encoded)

Epoch 0... Epoch 1... Epoch 2... Epoch 3... Epoch 4... Epoch 5... Epoch 6... Epoch 7... Epoch 8... Epoch 9... Epoch 10... Epoch 11... Epoch 12... Epoch 13... Epoch 14... Epoch 15... Epoch 16... Epoch 17... Epoch 18... Epoch 19... Epoch 20... Epoch 21... Epoch 22... Epoch 23... Epoch 24... Epoch 25... Epoch 26... Epoch 27... Epoch 28... Epoch 29... Epoch 30... Epoch 31... Epoch 32... Epoch 33... Epoch 34... Epoch 35... Epoch 36... Epoch 37... Epoch 38... Epoch 39... Epoch 40... Epoch 41... Epoch 42... Epoch 43... Epoch 44... Epoch 45... Epoch 46... Epoch 47... Epoch 48... Epoch 49... Epoch 50... Epoch 51... Epoch 52... Epoch 53... Epoch 54... Epoch 55... Epoch 56... Epoch 57... Epoch 58... Epoch 59... Epoch 60... Epoch 61... Epoch 62... Epoch 63... Epoch 64... Epoch 65... Epoch 66... Epoch 67... Epoch 68... Epoch 69... Epoch 70... Epoch 71... Epoch 72... Epoch 73... Epoch 74... Epoch 75... Epoch 76... Epoch 77... Epoch 78... Epoch 79... Epoch 80... Epoch 81... Epoch 82... Epoch 83... Ep

In [45]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([
    ('processor', preprocessor), #Step1 - preprocess data
    ('clf', LogisticRegression()) #step2 - classifier (default : LogisticRegression )
])

from sklearn.model_selection import cross_validate

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clfs = []
clfs.append(myclf)
# clfs.append(LogisticRegression())
# clfs.append(KNeighborsClassifier(n_neighbors=3))
# clfs.append(DecisionTreeClassifier())
# clfs.append(RandomForestClassifier())
# clfs.append(GradientBoostingClassifier())

from sklearn.model_selection import cross_val_score
for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_val_score(pipeline, x.values,y_encoded,cv=10, scoring="accuracy")
    print("===============")
    print(classifier)
    print(scores)

Epoch 0... Epoch 1... Epoch 2... Epoch 3... Epoch 4... Epoch 5... Epoch 6... Epoch 7... Epoch 8... Epoch 9... Epoch 10... Epoch 11... Epoch 12... Epoch 13... Epoch 14... Epoch 15... Epoch 16... Epoch 17... Epoch 18... Epoch 19... Epoch 20... Epoch 21... Epoch 22... Epoch 23... Epoch 24... Epoch 25... Epoch 26... Epoch 27... Epoch 28... Epoch 29... Epoch 30... Epoch 31... Epoch 32... Epoch 33... Epoch 34... Epoch 35... Epoch 36... Epoch 37... Epoch 38... Epoch 39... Epoch 40... Epoch 41... Epoch 42... Epoch 43... Epoch 44... Epoch 45... Epoch 46... Epoch 47... Epoch 48... Epoch 49... Epoch 50... Epoch 51... Epoch 52... Epoch 53... Epoch 54... Epoch 55... Epoch 56... Epoch 57... Epoch 58... Epoch 59... Epoch 60... Epoch 61... Epoch 62... Epoch 63... Epoch 64... Epoch 65... Epoch 66... Epoch 67... Epoch 68... Epoch 69... Epoch 70... Epoch 71... Epoch 72... Epoch 73... Epoch 74... Epoch 75... Epoch 76... Epoch 77... Epoch 78... Epoch 79... Epoch 80... Epoch 81... Epoch 82... Epoch 83... Ep

Epoch 89... Epoch 90... Epoch 91... Epoch 92... Epoch 93... Epoch 94... Epoch 95... Epoch 96... Epoch 97... Epoch 98... Epoch 99... Epoch 0... Epoch 1... Epoch 2... Epoch 3... Epoch 4... Epoch 5... Epoch 6... Epoch 7... Epoch 8... Epoch 9... Epoch 10... Epoch 11... Epoch 12... Epoch 13... Epoch 14... Epoch 15... Epoch 16... Epoch 17... Epoch 18... Epoch 19... Epoch 20... Epoch 21... Epoch 22... Epoch 23... Epoch 24... Epoch 25... Epoch 26... Epoch 27... Epoch 28... Epoch 29... Epoch 30... Epoch 31... Epoch 32... Epoch 33... Epoch 34... Epoch 35... Epoch 36... Epoch 37... Epoch 38... Epoch 39... Epoch 40... Epoch 41... Epoch 42... Epoch 43... Epoch 44... Epoch 45... Epoch 46... Epoch 47... Epoch 48... Epoch 49... Epoch 50... Epoch 51... Epoch 52... Epoch 53... Epoch 54... Epoch 55... Epoch 56... Epoch 57... Epoch 58... Epoch 59... Epoch 60... Epoch 61... Epoch 62... Epoch 63... Epoch 64... Epoch 65... Epoch 66... Epoch 67... Epoch 68... Epoch 69... Epoch 70... Epoch 71... Epoch 72... Ep

In [42]:
y_encoded

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,